<div style="text-align: center;">
<h1>Reel Realities: How Gender and Age Shape Success Across Box Office and Streaming Platforms</h1>
</div>

### <u>Imports</u>

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import ast
import re
import numpy as np

### 1. <u>Data cleaning and pre-processing</u>

#### 1.1 CMU Dataset

We will use three files of the CMU dataset:
1. "plot_summaries.txt" gives us the plots of the movies.
2. "movie.metadata.tsv" gives us information about the languages, countries, and genres of the movies.
3. "character.metadata.tsv" gives us 

In [ ]:
plot_summaries_df = pd.read_csv("data/CMU/plot_summaries.txt", delimiter="\t", names = ["Wikipedia_movie_ID", "Plot Summaries"])

print(f"The plot summaries dataframe has {len(plot_summaries_df):,} values.")

In [ ]:
column_names = ["Wikipedia_movie_ID",
                "Freebase_movie_ID",
                "Movie_name",
                "Movie_release_date",
                "Movie_box_office_revenue",
                "Movie_runtime",
                "Movie_languages",
                "Movie_countries",
                "Movie_genres"]

movie_metadata_df = pd.read_csv("data/CMU/movie.metadata.tsv", delimiter='\t', names = column_names)
print(f"The movie metadata dataframe has {len(movie_metadata_df):,} values.")

In [ ]:
movie_metadata_df["Movie_name"] = (movie_metadata_df["Movie_name"]
                    .str.strip()  #Remove leading/trailing whitespace
                    .str.lower()  #Convert to lowercase
                    .replace("", np.nan)  #Replace empty strings with NaN
                   )

In [ ]:
# Movie languages, countries, and genres are formatted as a dictionary, but they are actually a string.
# We convert them to comma spereated strings. 
movie_metadata_df["Movie_languages"] = movie_metadata_df["Movie_languages"].apply(ast.literal_eval)
movie_metadata_df["Movie_languages"] = movie_metadata_df["Movie_languages"].apply(lambda x: ', '.join(x.values()))

movie_metadata_df["Movie_countries"] = movie_metadata_df["Movie_countries"].apply(ast.literal_eval)
movie_metadata_df["Movie_countries"] = movie_metadata_df["Movie_countries"].apply(lambda x: ', '.join(x.values()))

movie_metadata_df["Movie_genres"] = movie_metadata_df["Movie_genres"].apply(ast.literal_eval)
movie_metadata_df["Movie_genres"] = movie_metadata_df["Movie_genres"].apply(lambda x: ', '.join(x.values()))

In [ ]:
#Cleaning
def clean_string_list(lst):
    # Check if lst is a list
    if isinstance(lst, list):
        return [s.strip().lower() if isinstance(s, str) and s.strip() != "" else np.nan for s in lst]
    elif isinstance(lst, str): 
        return lst.strip().lower()
    else:
        return np.nan 

movie_metadata_df["Movie_languages"] = movie_metadata_df["Movie_languages"].apply(clean_string_list)
movie_metadata_df["Movie_countries"] = movie_metadata_df["Movie_countries"].apply(clean_string_list)
movie_metadata_df["Movie_genres"] = movie_metadata_df["Movie_genres"].apply(clean_string_list)

In [ ]:
#There are inconsistencies in the date format: some rows have the date as year only, others as year-month, and some as year-month-day. We convert them all to the year format only since the month and day are not relevant to our analysis.
full_date_pattern = r'^\d{4}-\d{2}-\d{2}$' #Matches YYYY-MM-DD
year_month_pattern = r'^\d{4}-\d{2}$' #Matches YYYY-MM
year_only_pattern = r'^\d{4}$' #Matches YYYY

def identify_pattern(date):
    if pd.isna(date):
        return "Missing"
    elif re.match(full_date_pattern, date):
        return "Full Date (YYYY-MM-DD)"
    elif re.match(year_month_pattern, date):
        return "Year & Month Date (YYYY-MM)"
    elif re.match(year_only_pattern, date):
        return "Year Only (YYYY)"
    else:
        return "Other"

movie_metadata_df["Pattern"] = movie_metadata_df["Movie_release_date"].apply(identify_pattern)
pattern_summary = movie_metadata_df.groupby("Pattern").size().reset_index(name="Count")

print("Pattern Summary:")
print(pattern_summary)

In [ ]:
movie_metadata_df["Movie_release_date"] = movie_metadata_df["Movie_release_date"].apply(lambda x: str(x)[:4] if pd.notnull(x) else None)
movie_metadata_df = movie_metadata_df.drop(columns=["Pattern"])

In [ ]:
n_null_movie_metadata = ((movie_metadata_df.isnull().sum() / len(movie_metadata_df)) * 100).apply(lambda x: f"{x:,.2f}%")

print(
    f"Percentage of null values per column:\n"
    f"movie_metadata_df:\n{n_null_movie_metadata}"
)

In [ ]:
character_metadata = pd.read_csv("./data/CMU/character.metadata.tsv", delimiter="\t", header = None)
character_metadata.columns = [
    'Wikipedia_movie_ID', 
    'Freebase_movie_ID', 
    'Movie_release_date', 
    'Character_name', 
    'Actor_date_of_birth', 
    'Actor_gender', 
    'Actor_height', 
    'Actor_ethnicity', 
    'Actor_name', 
    'Actor_age_at_movie_release', 
    'Freebase_character_actor_map_ID', 
    'Freebase_character_ID', 
    'Freebase_actor_ID'
]

In [ ]:
character_metadata['Actor_name']= character_metadata['Actor_name'].fillna('unknown')
character_metadata["Actor_name"] = character_metadata["Actor_name"].str.lower()

# Ensures that the date columns are correctly interpreted as datetime objects in pandas. 
character_metadata['Movie_release_date'] = pd.to_datetime(character_metadata['Movie_release_date'], errors='coerce').dt.date
character_metadata['Actor_date_of_birth'] = pd.to_datetime(character_metadata['Actor_date_of_birth'], errors='coerce').dt.date

# Drop rows where 'Movie_release_date' or 'Actor_date_of_birth' is NaT
character_metadata.dropna(subset=['Movie_release_date', 'Actor_date_of_birth'], inplace=True)

# Calculate Actor_age_at_movie_release if missing and data is available
character_metadata['Actor_age_at_movie_release'] = character_metadata.apply(
    lambda row: (row['Movie_release_date'] - row['Actor_date_of_birth']).days // 365 if pd.isnull(row['Actor_age_at_movie_release']) else row['Actor_age_at_movie_release'],
    axis=1
)

#Drop unnecessary columns , heere the actors' heights
character_metadata.drop(columns=['Actor_height', 'Actor_ethnicity', 'Character_name'], inplace=True)


In [ ]:
print(f"Movies without actor gender data: {character_metadata['Actor_gender'].isnull().sum()}") 

We therefore drop the mvies with no information the gender of the actors.

In [ ]:
character_metadata.dropna(subset=['Actor_gender'], inplace=True)

Now, let's groupe the actors together by creating a *Cast* column on the *movie_metadata_df*. 

In [ ]:
cast_per_movie = character_metadata.groupby('Freebase_movie_ID')['Actor_name'].apply(lambda x: ', '.join(x)).reset_index()
cast_per_movie = cast_per_movie.rename(columns={'Actor_name': 'Cast'})

movie_metadata_df = movie_metadata_df.merge(cast_per_movie, on='Freebase_movie_ID', how='left')

In [ ]:
missing_cast_count = movie_metadata_df["Cast"].isna().sum()
total_movies = len(movie_metadata_df)
print(f"Movies without actor data: {missing_cast_count} out of {total_movies} ({missing_cast_count / total_movies * 100:.2f}%)")

We decide to drop the movies without actor data since it is one of the most important data of ur study. 

In [ ]:
movie_metadata_df = movie_metadata_df.dropna(subset = ['Cast'])

In [ ]:
# Create columns for male, female actor counts
actor_counts = character_metadata.groupby('Freebase_movie_ID')['Actor_gender'].value_counts().unstack(fill_value=0)
actor_counts = actor_counts.rename(columns={'M': 'Male_actors', 'F': 'Female_actors'})

# Ensure all columns are present (set missing columns to 0 if necessary)
for col in ['Male_actors', 'Female_actors']:
    if col not in actor_counts.columns:
        actor_counts[col] = 0

actor_counts = actor_counts.reset_index()

# Merge actor counts with the movie_metadata_df DataFrame
movie_metadata_df = movie_metadata_df.merge(actor_counts, on='Freebase_movie_ID', how='left')

# Fill NaN values with 0 
movie_metadata_df[['Male_actors', 'Female_actors']] = movie_metadata_df[['Male_actors', 'Female_actors']].fillna(0)

In [ ]:
# Calculate the percentage of female actors in each movie and round to two decimal places
movie_metadata_df['Female_actor_percentage'] = (
    (movie_metadata_df['Female_actors'] / 
     (movie_metadata_df['Female_actors'] + movie_metadata_df['Male_actors'])) * 100
).round(2)

#### 1.2 IMDB Dataset

We will use two IMDB datasets to decribe movies:
1. "title.ratings.tsv" gives us the ratings of the movies as voted by viewers. 
2. "title.basics.tsv", indexes into "title.ratings.tsv" using a alphanumeric unique identifier of the title. It gives general information about the movie such as runtime, release date and adult rating.
3. "title.crew.tsv", indexes into the previous two using the same alphanumeric unique identifier of the title. It gives information on the directors and writers of the movie.

Reference:
Internet Movie Database. (2024). IMDb non-commercial datasets. Retrieved from https://developer.imdb.com/non-commercial-datasets/

In [ ]:
# Loading the datasets. Null values are represented using "\N".
IMDB_ratings_df = pd.read_csv("data/IMDB/title.ratings.tsv", delimiter="\t", na_values="\\N", names=["tconst", "Average_ratings", "Num_votes"], low_memory= False)

column_names_basics = ["tconst",
                       "Title_type",
                       "Primary_title",
                       "Original_title",
                       "Is_adult",
                       "Start_year",
                       "End_year",
                       "Movie_runtime",
                       "Movie_genres"]
IMDB_basics_df = pd.read_csv("data/IMDB/title.basics.tsv", delimiter="\t", na_values="\\N", low_memory=False, names=column_names_basics)

IMDB_crew_df = pd.read_csv("data/IMDB/title.crew.tsv", delimiter="\t", na_values="\\N", low_memory=False, names=["tconst", "Directors", "Writers"])

In [ ]:
# Checking the lengths of the datasets
print(f"Length of IMDB_ratings_df: {len(IMDB_ratings_df):,}\n"
      f"Length of IMDB_basics_df: {len(IMDB_basics_df):,}\n"
      f"Length of IMDB_crew_df: {len(IMDB_crew_df):,}")

Before dealing with the null values we will merge the dataframes together using the alphanumeric unique identifier.

In [ ]:
# Merging all three datasets.
IMDB_merged_df = pd.merge(IMDB_ratings_df, IMDB_basics_df, how="inner", left_on="tconst", right_on="tconst")
IMDB_merged_df = pd.merge(IMDB_merged_df, IMDB_crew_df, how="inner", on="tconst")

print(f"The resulting merged dataframe has length: {len(IMDB_merged_df):,}.")
print(f"{len(IMDB_ratings_df)-len(IMDB_merged_df):,} rows were lost in the merging process.")

We can see we do not lose a lot of rows with respect to the IMDB_ratings_df dataframe.

Next, we look at titleType. These dataframes do not only have movies but also short movies, tv shows, episodes. The next step is thus to filter only movies.

In [ ]:
# Filtering movies from the list of titles.
IMDB_merged_df = IMDB_merged_df[IMDB_merged_df["Title_type"] == "movie"]

print(f"There are {len(IMDB_merged_df):,} movies in the resulting dataframe.")

Formatting the strings appropriately.

In [ ]:
IMDB_merged_df["Original_title"] = IMDB_merged_df["Original_title"].apply(clean_string_list)
IMDB_merged_df["Primary_title"] = IMDB_merged_df["Primary_title"].apply(clean_string_list)

We can now look at null values in the merged IMDB dataframe.

In [ ]:
# Checking the percentage of null values in the dataset.
n_null_IMDB = ((IMDB_merged_df.isnull().sum() / len(IMDB_merged_df)) * 100).apply(lambda x: f"{x:,.2f}%")

print(
    f"Percentage of null values per column:\n"
    f"IMDB_ratings_df:\n{n_null_IMDB}"
)

The end year is always missing. Other than that the proportion of missing values is very small (< 12%). End year does not have any useful information for our intended analysis and can thus be dropped. We can also drop the titleType column since we know they are all movies after the filtering that was done above.

In [ ]:
# Dropping unnecessary columns.
IMDB_merged_df = IMDB_merged_df.drop(columns=["End_year", "Title_type"], axis=1)

print(f"The resulting dataframe has {len(IMDB_merged_df):,} rows.")

We will use ratings from the IMDB dataset during our study. However, these ratings are based on viewer votes. Initially we thought of discarding rows with too few votes. However, there could be a link between number of votes and number of views of a movie (although definitely not a direct one). We thus decided to keep all rows for the analysis.

#### 1.3 Merging the datasets

##### 1.3.1 Merging IMDB and CMU Movies

In [ ]:
# Merging on the original title.
merge1 = pd.merge(IMDB_merged_df, movie_metadata_df, how="inner", left_on="Original_title", right_on="Movie_name")
# Merging on the primary title.
merge2 = pd.merge(IMDB_merged_df, movie_metadata_df, how="inner", left_on="Primary_title", right_on="Movie_name")

# Concatenating and dropping duplicates that appear from movies with the same Original_title and Primary_title.
movie_df = pd.concat([merge1, merge2]).drop_duplicates(subset=["tconst"]).reset_index(drop=True)

print(f"The resulting dataframe has {len(movie_df):,} rows.")

Some columns appear twice. Let's take a look at the proportion of null values in each duplicate column.

In [ ]:
n_null_movie = (movie_df.isnull().sum()/len(movie_df)).apply(lambda x:f"{x:.2%}")

print(f"Percentage of null values per column:\n{n_null_movie}")

We can see:
- Movie_runtime_x and Movie_runtime_y have 6.02% and 11.58% missing values respectively. We will combine the non null values from both these columns into a new column called runtimeMinutes and then drop the previous two columns. 
- Movie_genres_x has 1.54% missing values against 0.00% missing values for Movie_genres_y. Furthermore Movie_genres_y is from the CMU dataset and seems more complete. We will thus drop the genres_x column.

In [ ]:
# Combining all non-null values from Movie_runtime_x and Movie_runtime_y into runtimeMinutes.
movie_df["Movie_runtime"] = movie_df["Movie_runtime_x"].combine_first(movie_df["Movie_runtime_y"])

# Dropping the unnecessary columns.
movie_df.drop(columns=["Movie_runtime_x", "Movie_runtime_y", "Movie_genres_x"], inplace=True)

Let's now see if Movie_name, originalTitle and primaryTitle are all necessary or if there are any redundancies.

In [ ]:
test1 = movie_df["Movie_name"] == movie_df["Primary_title"]  
test2 = movie_df["Movie_name"] == movie_df["Original_title"]

# Checking if there are any movies for which Movie_name is not either in Primary_title or Original_title
print(f"There are {(~(test1 | test2)).sum().item()} movies for which Movie_name is in neither Primary_title ot Original_title.")

We can see the Movie_name column is redundant as its information is either in primaryTitle or in originalTitle. We can thus drop this column.

In [ ]:
movie_df.drop(columns="Movie_name", inplace=True)

We can now also drop movie identifier columns (as everything is already indexed): tconst, Freebase_movie_ID and Wikipedia_movie_ID.

In [ ]:
movie_df.drop(columns=["Wikipedia_movie_ID", "tconst", "Freebase_movie_ID"], inplace=True)

##### 1.3.3 Merging Kaggle and The numbers datasets to the rest

We have a problem, a lot of missing values(NaN or 0) in Movie box office revenue in movie_df and no production budget included. 
These two new datasets include information about movies revenue and budget and other attributes which are crucial for our analysis.

- `Kaggle_df`: Contains important details about movies, including budgets, genres, and revenue.
- `TheNumbers_df`: Provides detailed financial data, focusing on budget and box office gross both domestically and worldwide.

In [ ]:
Kaggle_df = pd.read_csv("data/KaggleSingh/movie_dataset.csv", na_values="\\N", low_memory=False)
TheNumbers_df = pd.read_csv("data/TheNumbers/tn.movie_budgets.csv", na_values="\\N", low_memory=False)

Quickly view the structure of the dataframes to understand the data types and confirm data has loaded correctly

In [ ]:
Kaggle_df.head()

In [ ]:
TheNumbers_df.head()

We now reduce the complexity of the Kaggle dataset by retaining only the columns necessary for our analysis. Additionally, we clean and convert financials of both datasets to float for consistency with the movie_df. 


In [ ]:
# Dropping the unnecessary columns.
Kaggle_df = Kaggle_df[["budget", "genres", "original_title", "popularity", "production_companies", "production_countries", "revenue"]]
Kaggle_df["budget"] = Kaggle_df["budget"].astype(float)
Kaggle_df["revenue"] = Kaggle_df["revenue"].astype(float)


In [ ]:
Kaggle_df.head()

In [ ]:
# Method to remove '$' and ',' from the financials
def clean_currency_column(column):
    return column.str.replace('[^\d.]', '', regex=True).astype(float)

TheNumbers_df["production_budget"] = clean_currency_column(TheNumbers_df["production_budget"])
TheNumbers_df["domestic_gross"] = clean_currency_column(TheNumbers_df["domestic_gross"])
TheNumbers_df["worldwide_gross"] = clean_currency_column(TheNumbers_df["worldwide_gross"])

In [ ]:
TheNumbers_df.head()

We now perform an outer merge to include all records from both datasets, aligning them based on movie titles.


In [ ]:
# Normalize the title columns for case insensitive matching
Kaggle_df["original_title"] = Kaggle_df["original_title"].str.strip().str.lower()
TheNumbers_df["movie"] = TheNumbers_df["movie"].str.strip().str.lower()

# Merge both dataframes on the title columns
merged_df = pd.merge(Kaggle_df, TheNumbers_df, left_on="original_title", right_on="movie", how='outer')
merged_df.head()


##### Post-merge cleaning

We create the main financial columns ('movie_budget' and 'movie_revenue') by selecting non-null and non-zero values, prioritizing data from TheNumbers dataset when available.


In [ ]:
merged_df['movie_budget'] = np.where((merged_df['production_budget'].notna() & (merged_df['production_budget'] != 0)),
                                     merged_df['production_budget'],
                                     merged_df['budget'])

merged_df['movie_revenue'] = np.where((merged_df['worldwide_gross'].notna() & (merged_df['worldwide_gross'] != 0)),
                                      merged_df['worldwide_gross'],
                                      merged_df['revenue'])

# Drop the original columns that were merged into new ones
merged_df.drop(columns=['budget', 'revenue', 'production_budget', 'worldwide_gross', 'movie'], inplace=True)

merged_df.head()


Filter out movies where both budget and revenue data are missing or zero, as they do not provide value for financial analysis.


In [ ]:
# Create a mask where both columns are either zero or NaN
mask = (
    (merged_df["movie_budget"].isna() | (merged_df["movie_budget"] == 0)) &
    (merged_df["movie_revenue"].isna() | (merged_df["movie_revenue"] == 0))
)

# Apply the mask and drop those rows
df_filtered = merged_df[~mask]

In [ ]:
print("The number of available and relevant movie budget information is " + str(df_filtered["movie_budget"].size))
print("The number of available and relevant movie revenue information is " + str(df_filtered["movie_revenue"].size))

df_filtered.head()

Two merging strategies are employed here to maximize the potential for matching records across the datasets:

Original Title Merge: Merging based on the Original_title from movie_df to the original_title from df_filtered to catch the most direct title matches. 

Secondary Title Merge: Using Primary_title as an alternative matching criterion, knowing that some movies might be listed under alternate titles.


In [ ]:
# Normalize the title columns for case-insensitive matching
movie_df["Original_title"] = movie_df["Original_title"].str.strip().str.lower()
movie_df["Primary_title"] = movie_df["Primary_title"].str.strip().str.lower()

# Merging on the original title.
merge1 = pd.merge(movie_df, df_filtered, how="left", left_on="Original_title", right_on="original_title")
# Merging on the secondary title.
merge2 = pd.merge(movie_df, df_filtered, how="left", left_on="Primary_title", right_on="original_title")

# Concatenating and dropping duplicates that appear from movies with the same Original_title and Primary_title.
final_df = pd.concat([merge1, merge2]).drop_duplicates().reset_index(drop=True)

# Filter out movies where revenue data is missing or zero and prioritize initial revenue
final_df["final_movie_revenue"] = np.where(
    (final_df["Movie_box_office_revenue"].notna() & (final_df["Movie_box_office_revenue"] != 0)),
    final_df["Movie_box_office_revenue"],
    final_df["movie_revenue"]
)

final_df.drop(columns=["Movie_box_office_revenue", "movie_revenue", "original_title", "genres", "id", "release_date"], inplace=True, errors='ignore')

# Renaming the columns to follow the convention
rename_columns = {"popularity":"Popularity", 
                  "production_companies":"Production_companies", 
                  "prodcution_countries":"Production_countries",
                  "domestic_gross":"Domestic_gross",
                  "movie_budget":"Movie_budget",
                  "final_movie_revenue":"Final_movie_revenue"}

final_df.rename(columns=rename_columns, inplace=True)


print(f"The resulting dataframe has {len(final_df):,} rows.")
final_df.head()

### 2. <u>Our success metric</u>

We decided to add an ROI(Return on investment) column which will help us after with our financial success analysis

In [ ]:
# Calculating ROI column based on profits and budget
final_df["ROI"] = ((final_df["Final_movie_revenue"] - final_df["Movie_budget"]) / final_df["Movie_budget"]) 

Review the processed dataset and summarize the null values to ensure data integrity before proceeding to the analysis phase.


In [ ]:
n_null_movie = (final_df.isnull().sum()/len(final_df)).apply(lambda x:f"{x:.2%}")

print(f"Percentage of null values per column:\n{n_null_movie}")

### 3. <u>Gender and age vs success BLABLABLA</u>

Dependent variables:
- Ratings
- Success metric
- Profit ratio

Independent variables:
- Gender
- Age
- Genre
- isAdult?
- Movie country
- Movie language
- Release date

Look at adding starpower

### 4. <u>How does it compare to streaming platforms? Are movies made for these platforms different? Have box office movies adapted since the rise of streaming?</u>

### 5. <u>What are the social reasons behind the presence of female characters in movies? Is it due to sexualization or genuine equality of representation?</u>